In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
import numpy as np, json, os

# 1️⃣ Load your model
model = SentenceTransformer("intfloat/multilingual-e5-large")

# 2️⃣ Load article-based dataset
with open(r"C:\Users\PC\OneDrive\Desktop\chatbot\data\processed\penal_code_articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)

In [2]:

# 3️⃣ Encode all articles into vectors
embeddings = model.encode(articles, show_progress_bar=True)
embeddings = np.array(embeddings)
print("✅ Embeddings shape:", embeddings.shape)

# 4️⃣ Save them for future use
os.makedirs(r"C:\Users\PC\OneDrive\Desktop\chatbot\data\embeddings\articles", exist_ok=True)
np.save(r"C:\Users\PC\OneDrive\Desktop\chatbot\data\embeddings\articles\penal_code_articles_e5.npy", embeddings)

print("✅ Saved article embeddings successfully!")

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

✅ Embeddings shape: (691, 1024)
✅ Saved article embeddings successfully!


In [3]:
import re
# build new chroma database
# 1️⃣ Load the embeddings and articles
embeddings = np.load(r"C:\Users\PC\OneDrive\Desktop\chatbot\data\embeddings\articles\penal_code_articles_e5.npy")
with open(r"C:\Users\PC\OneDrive\Desktop\chatbot\data\processed\penal_code_articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)

# 2️⃣ Extract article numbers as metadata
article_re = re.compile(r"المادة\s*(\d+)")
metadatas = []
for art in articles:
    match = article_re.search(art)
    number = match.group(1) if match else None
    metadatas.append({"article": number})

print("✅ Example metadata:", metadatas[:5])

✅ Example metadata: [{'article': '1'}, {'article': '2'}, {'article': '3'}, {'article': '4'}, {'article': '5'}]


In [4]:
# 3️⃣ Initialize Chroma client & embedding function
embed_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large"
)

chroma_client = chromadb.PersistentClient(
    path=r"C:\Users\PC\OneDrive\Desktop\chatbot\data\chroma_db_articles"
)

# Delete old collection if re-running
try:
    chroma_client.delete_collection("penal_code_articles")
except Exception:
    pass

# Create new collection
collection = chroma_client.create_collection(
    name="penal_code_articles",
    embedding_function=embed_func
)

# 4️⃣ Add all articles + embeddings + metadata
ids = [f"article_{i}" for i in range(len(articles))]
collection.add(
    ids=ids,
    documents=articles,
    embeddings=embeddings.tolist(),
    metadatas=metadatas
)

print(f"✅ Added {len(articles)} articles to Chroma (with metadata).")

✅ Added 691 articles to Chroma (with metadata).


In [5]:
query = "ما هي العقوبة على السرقة؟"
results = collection.query(
    query_texts=[query],
    n_results=3,
    include=["documents", "metadatas", "distances"]
)

for i, (doc, meta, dist) in enumerate(zip(results["documents"][0], results["metadatas"][0], results["distances"][0]), 1):
    sim = 1 - dist
    print(f"\n🔹 النتيجة {i} | المادة {meta.get('article')} | شبه ≈ {sim:.3f}")
    print(doc[:600])



🔹 النتيجة 1 | المادة 636 | شبه ≈ 0.847
المادة 636- معدلة وفقا للمرسوم الاشتراعي112 تاريخ16/9/ 1983 والقانون239 تاريخ
27/5/ 1993
- السرقة، التي لم تحدد لها عقوبة خاصة بموجب أحد نصوص هذا القانون، يعاقب عليها بالحبس من
شهرين إلى ثلاث سنوات وبالغرامة من مئة ألف إلى أربعمائة ألف ليرة.
وتشدد هذه العقوبة وفقا للمادة 257 إذا ارتكبت السرقة في إحدى الحالات التالية:
1- في المعابد والأبنية المأهولة.

2- خرى أو في القطارات أو في السفن أو بنشل المارة أكان ذلك في الطرق أو في الأماكن العامة الأ
الطائرات أو غيرها من وسائل النقل.
3- بفعل موظف أنيط به حفظ الأمن أو الحراسة حتى وإن ارتكبت السرقة في غير أوقات الدوام.
4- بفعل خادم مأجور يسرق مال مخدومه أ

🔹 النتيجة 2 | المادة 639 | شبه ≈ 0.847
المادة 639- معدلة وفقا للقانون487 تاريخ8/12/ 1995
يعاقب بالأشغال الشاقة من ثلاث سنوات إلى سبع سنوات كل من يرتكب السرقة في إحدى الحالات الآتية:
1- و الكسر في الأماكن المقفلة المصانة بالجدران، مأهولة أم غير مأهولة، أو يتسلقها في بواسطة الخلع أ
الداخل أو الخارج أو باستعمال المفاتيح المصنعة أو أي أداة أخرى أو بعد الدخول إ